In [26]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import functions as func

In [2]:
spark_session = SparkSession.builder.appName('Get most popular superheroes').getOrCreate()

In [25]:
data = spark_session.read.text('../../data/Marvel_Network/Marvel_Graph')
data.show(10)

+--------------------+
|               value|
+--------------------+
|5988 748 1722 375...|
|5989 4080 4264 44...|
|5982 217 595 1194...|
|5983 1165 3836 43...|
|5980 2731 3712 15...|
|5981 3569 5353 40...|
|5986 2658 3712 26...|
|5987 2614 5716 17...|
|5984 590 4898 745...|
|5985 3233 2254 21...|
+--------------------+
only showing top 10 rows



In [30]:
connections = data.withColumn('id', func.split(func.col('value'), ' ')[0]) \
                  .withColumn('num_of_connections', func.size(func.split(func.col('value'), ' ')) - 1) \
                  .groupBy('id') \
                  .agg(func.sum('num_of_connections').alias('total_connections')) \
                  .sort('total_connections', ascending=False)

connections.show(10)

+----+-----------------+
|  id|total_connections|
+----+-----------------+
| 859|             1937|
|5306|             1745|
|2664|             1532|
|5716|             1429|
|6306|             1397|
|3805|             1389|
|2557|             1374|
|4898|             1348|
|5736|             1292|
| 403|             1283|
+----+-----------------+
only showing top 10 rows



In [35]:
def get_sup_name_dict():
    name_dict = {}
    with open('../../data/Marvel_Network/Marvel_Names', 'r', encoding='unicode_escape') as f:
        for line in f.readlines():
            fields = line.split(' ')
            name_dict[fields[0]] = ' '.join(fields[1:])
    
    return name_dict

In [36]:
sup_name_dict = spark_session.sparkContext.broadcast(get_sup_name_dict())
list(sup_name_dict.value.items())[850:860]

[('851', '"CANDY"\n'),
 ('852', '"CANE/"\n'),
 ('853', '"CANNONBALL/JACK PULV"\n'),
 ('854', '"CANNONBALL II/SAM GU"\n'),
 ('855', '"CANNONBALL/SAM GUTHR"\n'),
 ('856', '"CANTOR, VERA"\n'),
 ('857', '"CAPRICORN/WILLARD WE"\n'),
 ('858', '"CAPRICORN II/"\n'),
 ('859', '"CAPTAIN AMERICA"\n'),
 ('860', '"CAPTAIN AMERICA IV/S"\n')]

In [37]:
def look_up_sup_name(id):
    return sup_name_dict.value[id]

In [38]:
look_up_sup_name_UDF = func.udf(look_up_sup_name)

In [39]:
named_connections = connections.withColumn('Superhero_name', look_up_sup_name_UDF(func.col('id'))) \
                               .select('Superhero_name', 'total_connections')

named_connections.show(10)

+--------------------+-----------------+
|      Superhero_name|total_connections|
+--------------------+-----------------+
| "CAPTAIN AMERICA"\n|             1937|
|"SPIDER-MAN/PETER...|             1745|
|"IRON MAN/TONY ST...|             1532|
|"THING/BENJAMIN J...|             1429|
|"WOLVERINE/LOGAN "\n|             1397|
|"MR. FANTASTIC/RE...|             1389|
|"HUMAN TORCH/JOHN...|             1374|
|"SCARLET WITCH/WA...|             1348|
|"THOR/DR. DONALD ...|             1292|
|"BEAST/HENRY &HAN...|             1283|
+--------------------+-----------------+
only showing top 10 rows



In [42]:
most_obscure_sups = named_connections.filter(named_connections.total_connections == 1)
most_obscure_sups.show(10)

+--------------------+-----------------+
|      Superhero_name|total_connections|
+--------------------+-----------------+
|    "BERSERKER II"\n|                1|
|          "BLARE/"\n|                1|
|"MARVEL BOY II/MA...|                1|
|"MARVEL BOY/MARTI...|                1|
|  "GIURESCU, RADU"\n|                1|
|   "CLUMSY FOULUP"\n|                1|
|          "FENRIS"\n|                1|
|          "RANDAK"\n|                1|
|       "SHARKSKIN"\n|                1|
| "CALLAHAN, DANNY"\n|                1|
+--------------------+-----------------+
only showing top 10 rows



In [45]:
connections_view = named_connections.createOrReplaceTempView("connections")

least_connected_sups = spark_session.sql('''
    SELECT * 
    FROM connections 
    WHERE total_connections = (
        SELECT MIN(total_connections)
        FROM connections
    )
''')
least_connected_sups.show()

+--------------------+-----------------+
|      Superhero_name|total_connections|
+--------------------+-----------------+
|    "BERSERKER II"\n|                1|
|          "BLARE/"\n|                1|
|"MARVEL BOY II/MA...|                1|
|"MARVEL BOY/MARTI...|                1|
|  "GIURESCU, RADU"\n|                1|
|   "CLUMSY FOULUP"\n|                1|
|          "FENRIS"\n|                1|
|          "RANDAK"\n|                1|
|       "SHARKSKIN"\n|                1|
| "CALLAHAN, DANNY"\n|                1|
|     "DEATHCHARGE"\n|                1|
|            "RUNE"\n|                1|
|     "SEA LEOPARD"\n|                1|
|     "RED WOLF II"\n|                1|
|          "ZANTOR"\n|                1|
|"JOHNSON, LYNDON ...|                1|
|      "LUNATIK II"\n|                1|
|            "KULL"\n|                1|
|"GERVASE, LADY AL...|                1|
+--------------------+-----------------+



In [50]:
min_connections = named_connections.agg(func.min('total_connections')).first()[0]
least_connected_sups = named_connections.filter(named_connections.total_connections == min_connections)

least_connected_sups.show(10)

+--------------------+-----------------+
|      Superhero_name|total_connections|
+--------------------+-----------------+
|    "BERSERKER II"\n|                1|
|          "BLARE/"\n|                1|
|"MARVEL BOY II/MA...|                1|
|"MARVEL BOY/MARTI...|                1|
|  "GIURESCU, RADU"\n|                1|
|   "CLUMSY FOULUP"\n|                1|
|          "FENRIS"\n|                1|
|          "RANDAK"\n|                1|
|       "SHARKSKIN"\n|                1|
| "CALLAHAN, DANNY"\n|                1|
+--------------------+-----------------+
only showing top 10 rows



In [51]:
spark_session.stop()